# Shark League Metric 

## Import Python libraries

In [9]:
import pandas as pd
import logging
import swc_simple_client as swc

## Configure logging

In [10]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename='shark_notebook.log',  
    level=logging.INFO,  
)

## Setup notebook variables

In [15]:
base_url = "https://analytics-project-em0g.onrender.com"


In [16]:
# Debug the API response first
week_api_response = swc.call_api_endpoint(base_url, swc.LIST_WEEKS_ENDPOINT)
print(f"Status code: {week_api_response.status_code}")
print(f"Response text: {week_api_response.text}")
print(f"Response headers: {dict(week_api_response.headers)}")

Status code: 200
Response text: [{"week_number":"202301","ppr_8_max_points":864.0,"ppr_10_max_points":1040.0,"ppr_12_max_points":1392.0,"ppr_14_max_points":1358.0,"half_ppr_8_max_points":888.0,"half_ppr_10_max_points":860.0,"half_ppr_12_max_points":1332.0,"half_ppr_14_max_points":1162.0,"std_8_max_points":848.0,"std_10_max_points":970.0,"std_12_max_points":1260.0,"std_14_max_points":1428.0,"last_changed_date":"2024-09-03"},{"week_number":"202302","ppr_8_max_points":680.0,"ppr_10_max_points":1220.0,"ppr_12_max_points":1224.0,"ppr_14_max_points":1204.0,"half_ppr_8_max_points":744.0,"half_ppr_10_max_points":1030.0,"half_ppr_12_max_points":1416.0,"half_ppr_14_max_points":1792.0,"std_8_max_points":832.0,"std_10_max_points":970.0,"std_12_max_points":1260.0,"std_14_max_points":1680.0,"last_changed_date":"2024-09-03"},{"week_number":"202303","ppr_8_max_points":640.0,"ppr_10_max_points":960.0,"ppr_12_max_points":1368.0,"ppr_14_max_points":1540.0,"half_ppr_8_max_points":1008.0,"half_ppr_10_max_p

## Get Custom Max Scores
### Use endpoint: LIST_WEEKS_ENDPOINT 

In [17]:
week_api_response = swc.call_api_endpoint(base_url,swc.LIST_WEEKS_ENDPOINT)
weeks_df = pd.DataFrame(week_api_response.json())
weeks_df['year'] = weeks_df['week_number'].str.slice(0, 4).astype(int)
weeks_df['week'] = weeks_df['week_number'].str.slice(4, 6).astype(int)

weeks_df = weeks_df.query('week <= 14')

max_totals_grouped_df = weeks_df.groupby('year').agg(
    ppr_12_max_points=('ppr_12_max_points', 'sum'), 
    half_ppr_8_max_points=('half_ppr_8_max_points', 'sum'))

display(max_totals_grouped_df)

,ppr_12_max_points,half_ppr_8_max_points
year,,
2023,16716.0,12304.0


## Get League Scoring Type
### Use Endpoint: LIST_LEAGUES_ENDPOINT

In [18]:
league_api_response = swc.call_api_endpoint(base_url,swc.LIST_LEAGUES_ENDPOINT)
leagues_df = pd.DataFrame(league_api_response.json())
leagues_df = leagues_df.drop(columns=['teams','last_changed_date'])
display(leagues_df)

,league_id,league_name,scoring_type,league_size
0,5001,Pigskin Prodigal Fantasy League,PPR,12
1,5002,Recurring Champions League,Half-PPR,8
2,5003,AHAHFZZFFFL,Half-PPR,8
3,5004,Gridiron Gurus Fantasy League,PPR,12
4,5005,Best League Ever,PPR,12


## Get Regular Season Scoring Totals - By Team
### Use Endpoint: LIST_TEAMS_ENDPOINT

In [19]:
team_api_response = swc.call_api_endpoint(base_url,swc.LIST_TEAMS_ENDPOINT)

teams_parsed_df = pd.json_normalize(team_api_response.json(), 'weekly_scores', 
                                    ['league_id', 'team_id', 'team_name'])

teams_parsed_df['year'] = (teams_parsed_df['week_number']
                           .str.slice(0, 4).astype(int))
teams_parsed_df['week'] = (teams_parsed_df['week_number']
                           .str.slice(4, 6).astype(int))

#get only regular season teams
teams_regular_season_df = teams_parsed_df.query('week <= 14')

#get team season totals
team_totals_df = teams_regular_season_df.groupby(
    ['league_id', 'team_id', 'year'], as_index = False
    )['fantasy_points'].sum()

team_totals_df.head()

,league_id,team_id,year,fantasy_points
0,5001,1001,2023,1238.0
1,5001,1002,2023,1216.0
2,5001,1003,2023,1193.0
3,5001,1004,2023,1071.0
4,5001,1005,2023,958.0


## League Balance Score
### Using Coefficient of Variation (CV) of league regular season totals

In [20]:
league_stats_df = team_totals_df.groupby(['league_id','year']).agg(
    league_points_sum=('fantasy_points', 'sum'),
    league_points_mean=('fantasy_points', 'mean'),
    league_points_stdev=('fantasy_points', 'std'),
    league_balance_score=('fantasy_points', 
                          lambda x: (100 -(x.std() / x.mean()) * 100))
).reset_index()

display(league_stats_df.sort_values(by='league_balance_score', ascending=False))

,league_id,year,league_points_sum,league_points_mean,league_points_stdev,league_balance_score
1,5002,2023,9126.0,1140.750000,58.801725,94.845345
0,5001,2023,14052.0,1171.000000,128.872735,88.994643
3,5004,2023,13752.0,1146.000000,148.203177,87.067786
4,5005,2023,13313.0,1109.416667,145.786555,86.859170
2,5003,2023,8599.0,1074.875000,157.835753,85.315897


## League Juice Score
### Compare league scoring to max potential scoring

In [21]:
league_stats_with_league_max_df = (league_stats_df[
    ['league_id','year', 'league_points_sum','league_balance_score']]
               .merge(max_totals_grouped_df,left_on = 'year', right_on='year'))

combined_metrics_df = (leagues_df[
    ['league_id','league_name','scoring_type', 'league_size']]
    .merge(league_stats_with_league_max_df, 
           left_on = 'league_id', right_on = 'league_id'))

combined_metrics_df['league_juice_score'] = combined_metrics_df.apply(
    lambda row: (
        100 * (row['league_points_sum'] / row['ppr_12_max_points'])
        if (row['scoring_type'] == 'PPR' and row['league_size'] == 12) 
        else (
            100 * (row['league_points_sum'] / row['half_ppr_8_max_points'])
            if (row['scoring_type'] == 'Half-PPR' and row['league_size'] == 8) 
            else None
        )
    ),
    axis=1
)

combined_metrics_df = (combined_metrics_df.drop(
    columns=['scoring_type','league_size','league_points_sum'
             ,'ppr_12_max_points','half_ppr_8_max_points',] )
)
display(combined_metrics_df)

,league_id,league_name,year,league_balance_score,league_juice_score
0,5001,Pigskin Prodigal Fantasy League,2023,88.994643,84.063173
1,5002,Recurring Champions League,2023,94.845345,74.171001
2,5003,AHAHFZZFFFL,2023,85.315897,69.887841
3,5004,Gridiron Gurus Fantasy League,2023,87.067786,82.268485
4,5005,Best League Ever,2023,86.859170,79.642259


# Create Shark League Score
## Shark League Score = (2 * League Juice Score) + League Balance Score

In [22]:
combined_metrics_df['shark_league_score'] = combined_metrics_df.apply(
    lambda league: (2 * league['league_juice_score']) +  league['league_balance_score'],
    axis=1
)
display(combined_metrics_df.sort_values(by='shark_league_score', ascending=False))

,league_id,league_name,year,league_balance_score,league_juice_score,shark_league_score
0,5001,Pigskin Prodigal Fantasy League,2023,88.994643,84.063173,257.120989
3,5004,Gridiron Gurus Fantasy League,2023,87.067786,82.268485,251.604756
4,5005,Best League Ever,2023,86.859170,79.642259,246.143688
1,5002,Recurring Champions League,2023,94.845345,74.171001,243.187348
2,5003,AHAHFZZFFFL,2023,85.315897,69.887841,225.091580
